In [211]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.utils.data as data_utils
# from torchvision import datasets, transforms

import pandas as pd
import numpy as np
# import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import networkx as nx
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from math import radians

import warnings
warnings.filterwarnings('ignore')

In [212]:
# mta ridership - aggregated on 4hr intervals

mta = pd.read_csv('/Users/devashishkhulbe/Desktop/research/GNN-traffic/subway_2021_ridership.csv')
mta = mta[mta['flag'] == True]
# aggregate on daily 

mta['DATE_time'] = pd.to_datetime(mta['DATE_time'])
mta['DATE'] = mta['DATE_time'].dt.date
mta_rider = mta.groupby(by=['DATE', 'STATION'], as_index=False).sum()[['DATE', 'STATION', 'ridership']]

daily_rider = pd.pivot(data=mta_rider, index='DATE', columns=['STATION'], values='ridership')
daily_rider.head()

STATION,1 AV,103 ST,103 ST-CORONA,104 ST,110 ST,111 ST,116 ST,121 ST,125 ST,135 ST,...,WAKEFIELD/241,WALL ST,WHITLOCK AV,WILSON AV,WINTHROP ST,WOODHAVEN BLVD,WOODLAWN,WORLD TRADE CTR,YORK ST,ZEREGA AV
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-26,191.0,317.0,703.0,20.0,239.0,70.0,84.0,216.0,782.0,1.0,...,267.0,53.0,35.0,299.0,224.0,800.0,362.0,71.0,7309.0,44.0
2020-12-27,159.0,366.0,649.0,19.0,229.0,87.0,65.0,213.0,822.0,0.0,...,315.0,46.0,20.0,253.0,188.0,696.0,349.0,60.0,447.0,47.0
2020-12-28,310.0,662.0,953.0,36.0,399.0,145.0,139.0,407.0,1226.0,536.0,...,495.0,218.0,55.0,478.0,542.0,1079.0,663.0,173.0,867.0,100.0
2020-12-29,390.0,678.0,1002.0,54.0,368.0,157.0,157.0,420.0,1187.0,57.0,...,478.0,227.0,48.0,461.0,573.0,1430.0,686.0,178.0,896.0,123.0
2020-12-30,402.0,680.0,1169.0,50.0,407.0,143.0,177.0,445.0,1116.0,NaN,...,484.0,238.0,52.0,454.0,538.0,1195.0,696.0,173.0,894.0,121.0


In [213]:
embed = nx.readwrite.read_gpickle('embeddings64.data')

In [214]:
st_nodes = pd.read_csv('/Users/devashishkhulbe/Desktop/research/GNN-traffic/subway_locations_connections.csv')
st_nodes.head()

,origin_name,origin_id,origin_lat,origin_long,dest_name,dest_id,dest_lat,dest_long
0,1 AV,L06,40.730953,-73.981628,"['3 Av', 'Bedford Av']","['L05', 'L08']","[40.732849, 40.717304]","[-73.986122, -73.956872]"
1,103 ST,119,40.799446,-73.968379,"['96 St', 'Cathedral Pkwy']","['120', '118']","[40.793919, 40.803967]","[-73.972323, -73.966847]"
2,103 ST - CORONA PLAZA,706,40.749865,-73.862700,"['Junction Blvd', '111 St']","['707', '705']","[40.749145, 40.75173]","[-73.869527, -73.855334]"
3,104 ST,A63,40.681711,-73.837683,"['111 St', 'Rockaway Blvd']","['A64', 'A61']","[40.684331, 40.680429]","[-73.832163, -73.843853]"
4,110 ST,623,40.795020,-73.944250,"['116 St', '103 St']","['622', '624']","[40.798629, 40.7906]","[-73.941617, -73.947478]"


In [215]:
# keep only stations which have embeddings
st_nodes_clean = st_nodes.loc[st_nodes.origin_id.isin(list(embed.wv.key_to_index.keys()))]
daily_rider = daily_rider.loc[:, daily_rider.columns.isin(red_st.origin_name)]
daily_rider.fillna(0, inplace=True)
daily_rider.head()

STATION,1 AV,103 ST,104 ST,110 ST,111 ST,116 ST,121 ST,125 ST,135 ST,14 ST,...,UTICA AV,VAN SICLEN AV,WALL ST,WHITLOCK AV,WILSON AV,WINTHROP ST,WOODHAVEN BLVD,WOODLAWN,YORK ST,ZEREGA AV
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-26,191.0,317.0,20.0,239.0,70.0,84.0,216.0,782.0,1.0,414.0,...,286.0,431.0,53.0,35.0,299.0,224.0,800.0,362.0,7309.0,44.0
2020-12-27,159.0,366.0,19.0,229.0,87.0,65.0,213.0,822.0,0.0,346.0,...,350.0,410.0,46.0,20.0,253.0,188.0,696.0,349.0,447.0,47.0
2020-12-28,310.0,662.0,36.0,399.0,145.0,139.0,407.0,1226.0,536.0,773.0,...,575.0,679.0,218.0,55.0,478.0,542.0,1079.0,663.0,867.0,100.0
2020-12-29,390.0,678.0,54.0,368.0,157.0,157.0,420.0,1187.0,57.0,752.0,...,563.0,699.0,227.0,48.0,461.0,573.0,1430.0,686.0,896.0,123.0
2020-12-30,402.0,680.0,50.0,407.0,143.0,177.0,445.0,1116.0,0.0,756.0,...,577.0,731.0,238.0,52.0,454.0,538.0,1195.0,696.0,894.0,121.0


In [218]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [295]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.hidden_dim = hidden_dim

        self.layers = nn.Sequential(
            nn.Linear(input_dim, self.hidden_dim),
            nn.ReLU(),

            nn.Dropout(0.2),
            
            nn.Linear(self.hidden_dim, 32),
            nn.ReLU(),
    
            nn.Linear(32, 1))

    def forward(self, X):
        out = self.layers(X)
        return out


class nn1(nn.Module):
    def __init__(self, embed, input_data, hidden_dim=128):
        super().__init__()
#         self.g = embed
#         self.h = input_data
        self.mlp = MLP(128, 512)
#         self.i = output_stat
#         self.j = input_stat
#         self.hidden_dim = hidden_dim

    def forward(self, X, embed):
        
        input = embed
        
        out = self.mlp(input).reshape(1, -1)
        out = nn.functional.normalize(out, dim=-1)
        
        targets = X.to(device)
        result = torch.sum(out.mul(targets), dim=-1)
        
        return result

In [ ]:
epochs = 10000
all_r2 = []

for stat in daily_rider.columns[:50]:
    
    print(stat)
    col = list(daily_rider.columns)
    col.remove(stat)
    ds = daily_rider[col].shift()[1:]
    ds[stat] = daily_rider[stat].values[1:]
    ds = pd.DataFrame(ds.values, columns=list(ds.columns))
    ds.index = list(ds.index)
    
    indices = list(range(len(ds)))
    
#     train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=17)
    train_indices, test_indices = indices[:250], indices[250:]
    
    criterion = nn.HuberLoss(delta=20).to(device)
    
    train_target = torch.tensor(ds.loc[train_indices, stat])
    train_feat = torch.tensor(ds.loc[train_indices, col].values)
    test_target = torch.tensor(ds.loc[test_indices, stat].values)
    test_feat = torch.tensor(ds.loc[test_indices, col].values)
    
    a_embed = torch.tensor(embed.wv[st_nodes[st_nodes.origin_name==stat].origin_id.values])
    b_embed = torch.tensor(embed.wv[st_nodes.loc[st_nodes.origin_name.isin(col), 'origin_id'].values])
    all_emb = torch.cat([a.expand(b.shape[0], b.shape[1]), b], dim=1)
    
    for epoch in range(epochs):
    
        model = nn1(all_emb, train_feat).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        model.train()

        optimizer.zero_grad()

            # get the output from forward propogation of our model
        output = model(train_feat, all_emb)

        train_accuracy = r2_score(output.cpu().detach().numpy(), train_target.cpu().detach().numpy())
        loss = criterion(output, train_target)
        loss.backward(retain_graph=True)

        optimizer.step()
        model.eval()
        
        with torch.no_grad():
            test_output = model(test_feat, all_emb)
            test_accuracy = r2_score(test_output.cpu().detach().numpy(), test_target.cpu().detach().numpy())
        
        if epoch == 0:
            best_loss = loss
            best_output = output
            best_acc = train_accuracy
            best_test_acc = test_accuracy
            best_test_output = test_output
        else:
            if loss < best_loss:
                best_loss = loss
                best_output = output
                best_acc = train_accuracy
                best_test_acc = test_accuracy
                best_test_output = test_output

        if epoch == 0 or (epoch+1) % 5000 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Train Accuracy: {:.4f}'.format(best_acc.item()),
                  'test Accuracy: {:.4f}'.format(best_test_acc.item()),
                  'Loss: {:.8f}'.format(best_loss.item()),)
            
        if epoch+1 == 10000:
            all_r2.append(best_test_acc)

1 AV
Epoch: 0001 Train Accuracy: -13.8266 test Accuracy: -11.9766 Loss: 130387.88050065
Epoch: 5000 Train Accuracy: -0.6657 test Accuracy: -11.5948 Loss: 2350.91452834
Epoch: 10000 Train Accuracy: -0.2989 test Accuracy: -10.9050 Loss: 2107.52638251
103 ST
Epoch: 0001 Train Accuracy: -12.4319 test Accuracy: -10.9768 Loss: 123513.66729277
Epoch: 5000 Train Accuracy: -0.1843 test Accuracy: -23.7006 Loss: 2701.76181351
Epoch: 10000 Train Accuracy: -0.1843 test Accuracy: -23.7006 Loss: 2701.76181351
104 ST
Epoch: 0001 Train Accuracy: -13.5818 test Accuracy: -12.0255 Loss: 108103.57531703
Epoch: 5000 Train Accuracy: -0.7170 test Accuracy: -14.8244 Loss: 2569.62529185
Epoch: 10000 Train Accuracy: -0.3493 test Accuracy: -12.8888 Loss: 2465.88863830
110 ST
Epoch: 0001 Train Accuracy: -13.3452 test Accuracy: -12.7991 Loss: 91191.34862085
Epoch: 5000 Train Accuracy: -0.6570 test Accuracy: -10.9660 Loss: 2382.07234030
Epoch: 10000 Train Accuracy: -0.6570 test Accuracy: -10.9660 Loss: 2382.07234030

In [314]:
all_r2

[-17.505170365488908]

In [312]:
best_test_acc

-11.726995366515707